In [1]:
# Make sure your Jupyter notebook is inside the virtualenv

# ! pip3 install pandas==1.2.5 numpy==1.20.3 -I -t .
# ! pip3 install gensim pyLDAvis
# ! pip3 install nltk

In [2]:
import os
import pandas as pd
from nltk.tokenize import RegexpTokenizer

In [3]:
os.chdir(os.path.dirname(os.path.realpath('__file__')))

In [4]:
liduo = pd.read_excel('利空与利多信息汇总.xlsx', sheet_name='利好信息')
likong = pd.read_excel('利空与利多信息汇总.xlsx', sheet_name=1)
merged = pd.read_excel('利空与利多信息汇总.xlsx', sheet_name='Merged')
liduo = liduo['Explanation'].tolist()
likong = likong['Explanation'].tolist()
merged = merged['Explanation'].tolist()
tokenizer = RegexpTokenizer(r'\w+')

In [5]:
merged

['I work at 02199 so I know some internal information',
 'According to reliable inside information: there are several other large shareholders with strong capital manipulating Xindu.',
 'Absolutely major inside news - Xindu Hotel is about to go up big starting tomorrow ！！！！！！！！！！！！！！！！',
 "I once promised someone to shut up for a week. Today the time is up. There is no impermeable wall, and today I looked, it seems that some news still leaked out. I swear I didn't say this, I wouldn't say something so unqualified. It's very meaningless if a person can read it. Here again I salute all the teachers and students, you are too great. Throughout the ages, there have been many famous and ruthless fighters, but not many of them have been able to make people respect them. Recently, the southern department of the double ha finally have a solution to the idea, things have changed, too much regret. Too much bookishness is not a good thing, but the good thing is that your team is very strong in gen

In [6]:
# 分词
liduo = [tokenizer.tokenize(liduo[i]) for i in range(len(liduo))]
likong = [tokenizer.tokenize(likong[i]) for i in range(len(likong))]
merged = [tokenizer.tokenize(merged[i]) for i in range(len(merged))]

In [40]:
# 删除数值，删除停用词
def word_clean(sentence, stop_words):
    sentence = [i.lower() for i in sentence]
    sentence = [token for token in sentence if not token.isnumeric()]
    sentence = [j for j in sentence if j not in stop_words]
    return sentence

# stop_words = open('stopwords.txt', 'r')
# stop_words = stop_words.read().split('\n')

stop_words = pd.read_csv('stopwords.txt', header=None)[0].to_list()


liduo = [word_clean(sentence, stop_words) for sentence in liduo]
likong = [word_clean(sentence, stop_words) for sentence in likong]
merged = [word_clean(sentence, stop_words) for sentence in merged]
#list(map(word_clean, likong))

In [43]:
merged

[['internal'],
 ['reliable',
  'inside',
  'shareholder',
  'strong',
  'capital',
  'manipulating',
  'xindu'],
 ['absolutely', 'major', 'inside', 'xindu', 'hotel', 'starting', 'tomorrow'],
 ['promised',
  'shut',
  'week',
  'impermeable',
  'wall',
  'looked',
  'leaked',
  'swear',
  'didn',
  'wouldn',
  'unqualified',
  'meaningless',
  'person',
  'read',
  'salute',
  'teacher',
  'student',
  'age',
  'famous',
  'ruthless',
  'fighter',
  'people',
  'respect',
  'southern',
  'department',
  'double',
  'ha',
  'finally',
  'solution',
  'idea',
  'changed',
  'regret',
  'bookishness',
  'team',
  'strong',
  'mystery',
  'advance',
  'ability',
  'control',
  'plate',
  'average',
  'smooth',
  'congratulate',
  'advance',
  'detail',
  'speculation',
  'judgment',
  'evaluation',
  'operation',
  'economic',
  'organization',
  'carry',
  'history',
  'era',
  'zhongtang',
  'merchant',
  'bureau',
  'mao',
  'resource',
  'design',
  'international',
  'cooperation',
  '

In [8]:
import nltk
# nltk.download('wordnet')

In [9]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
liduo = [[lemmatizer.lemmatize(token) for token in doc] for doc in liduo]
likong = [[lemmatizer.lemmatize(token) for token in doc] for doc in likong]
merged = [[lemmatizer.lemmatize(token) for token in doc] for doc in merged]

In [10]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(liduo, min_count=1)
liduo = [bigram[lst] for lst in liduo]

bigram = Phrases(likong, min_count=1)
likong = [bigram[lst] for lst in likong]

bigram = Phrases(merged, min_count=1)
merged = [bigram[lst] for lst in merged]

In [11]:
# Remove rare and common tokens.
from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
liduo_dictionary = Dictionary(liduo)

# Filter out words that occur less than 2 documents, or more than 60% of the documents.
liduo_dictionary.filter_extremes(no_below=1, no_above=0.6)

# Create a dictionary representation of the documents.
likong_dictionary = Dictionary(likong)

# Filter out words that occur less than 2 documents, or more than 60% of the documents.
likong_dictionary.filter_extremes(no_below=1, no_above=0.6)

# Create a dictionary representation of the documents.
merged_dictionary = Dictionary(merged)

# Filter out words that occur less than 2 documents, or more than 60% of the documents.
merged_dictionary.filter_extremes(no_below=1, no_above=0.6)

In [12]:
# Bag-of-words representation of the documents.
liduo_corpus = [liduo_dictionary.doc2bow(doc) for doc in liduo]
likong_corpus = [likong_dictionary.doc2bow(doc) for doc in likong]
merged_corpus = [merged_dictionary.doc2bow(doc) for doc in merged]

In [13]:
# Train liduo model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 5
chunksize = 40
passes = 5
iterations = 200
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = liduo_dictionary[0]  # This is only to "load" the dictionary.
id2word = liduo_dictionary.id2token

liduo_model = LdaModel(
    corpus=liduo_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [14]:
# Train likong model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 5
chunksize = 40
passes = 5
iterations = 200
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = likong_dictionary[0]  # This is only to "load" the dictionary.
id2word = likong_dictionary.id2token

likong_model = LdaModel(
    corpus=likong_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [21]:
# Train merged model.
from gensim.models import LdaModel

# Set training parameters.
num_topics = 5
chunksize = 20
passes = 5
iterations = 200
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = merged_dictionary[0]  # This is only to "load" the dictionary.
id2word = merged_dictionary.id2token

merged_model = LdaModel(
    corpus=merged_corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [22]:
liduo_model.top_topics(liduo_corpus)

[([(0.052356414, 'product'),
   (0.018754564, 'real_estate'),
   (0.014037454, 'investment'),
   (0.011058468, 'expected_return'),
   (0.009613602, 'capital'),
   (0.009482235, 'bank'),
   (0.007999663, 'project'),
   (0.007992011, 'stock'),
   (0.007965692, 'fund'),
   (0.007950622, 'investor'),
   (0.007941884, 'customer'),
   (0.0066348384, 'inside'),
   (0.0064681834, 'strong'),
   (0.006450244, 'financing'),
   (0.0064418907, 'wealth_management'),
   (0.0064358064, 'risk'),
   (0.0049731503, 'source'),
   (0.004970341, 'head'),
   (0.0049644834, 'starting'),
   (0.0049327784, 'institution')],
  -2.6377021457210748),
 ([(0.016845038, 'industry'),
   (0.0135160005, 'product'),
   (0.012471392, 'investment'),
   (0.0108428905, 'hong_kong'),
   (0.008686306, 'financial'),
   (0.007330978, 'medium'),
   (0.0061759218, 'negative'),
   (0.0061522024, 'flavor'),
   (0.0061284346, 'mainland'),
   (0.005211385, 'investor'),
   (0.0051914128, 'stock'),
   (0.0049816323, 'plasticizer'),
   (0

In [23]:
likong_model.top_topics(likong_corpus)

[([(0.03498369, 'banker'),
   (0.023698652, 'stock'),
   (0.01805621, 'yesterday'),
   (0.018056111, 'teacher'),
   (0.012413772, 'fund'),
   (0.012413543, 'chip'),
   (0.012412328, 'share'),
   (0.0068451706, 'chemical'),
   (0.0068427674, 'author'),
   (0.0068195187, 'original_post'),
   (0.0068194363, 'internal'),
   (0.0067714322, 'lobbyist'),
   (0.0067711812, 'shanghai'),
   (0.0067710583, 'opinion'),
   (0.0067710415, 'term'),
   (0.006770981, 'optimistic'),
   (0.006770977, 'suppressed'),
   (0.0067709708, 'local'),
   (0.0067709573, 'friend'),
   (0.0067709554, 'backhand')],
  -2.354577953805337),
 ([(0.031007266, 'shantou_lianmei'),
   (0.013781032, 'report'),
   (0.013780306, 'company'),
   (0.009474551, 'shareholder'),
   (0.009474402, 'yaqian'),
   (0.009474393, 'profile'),
   (0.009474375, 'son_zhuang'),
   (0.0094743725, 'lianmei'),
   (0.009474366, 'capital'),
   (0.00947436, 'acquisition'),
   (0.009474357, 'mother_son'),
   (0.009474354, 'family'),
   (0.0094743455, '

In [24]:
merged_model.top_topics(merged_corpus)

[([(0.017417463, 'real_estate'),
   (0.016001452, 'stock'),
   (0.015404894, 'banker'),
   (0.010658285, 'fund'),
   (0.009790023, 'capital'),
   (0.009427221, 'share'),
   (0.009230509, 'expected_return'),
   (0.007954616, 'bank'),
   (0.0069400696, 'shareholder'),
   (0.0068739825, 'internal'),
   (0.006677904, 'customer'),
   (0.0065655075, 'project'),
   (0.005400278, 'wealth_management'),
   (0.005400278, 'financing'),
   (0.005400278, 'risk'),
   (0.004954765, 'buy'),
   (0.0048280726, 'strong'),
   (0.004592718, 'month'),
   (0.004451535, 'chip'),
   (0.004248683, 'rise')],
  -7.91283300935473),
 ([(0.014054629, 'shantou_lianmei'),
   (0.012709061, 'product'),
   (0.00866686, 'medium'),
   (0.00807487, 'shipment'),
   (0.00807487, 'sector_dragon'),
   (0.0068709166, 'report'),
   (0.0068310606, 'financial'),
   (0.0063513555, 'hong_kong'),
   (0.006158398, 'profile'),
   (0.0061192582, 'brother'),
   (0.005920636, 'insider'),
   (0.0056503876, 'share'),
   (0.0052691456, 'shangh

In [28]:
import warnings
warnings.filterwarnings('ignore')

# 利多结果展示

## Visualization

In [29]:
import pyLDAvis
from pyLDAvis import gensim_models
pyLDAvis.enable_notebook()
vis = gensim_models.prepare(liduo_model, liduo_corpus, dictionary=liduo_dictionary)
vis
# import pyLDAvis.gensim_models as gensimvis
# gensimvis.prepare(liduo_model, liduo_corpus, dictionary=liduo_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.092428  0.093187       1        1  44.978450
3      0.153019  0.035842       2        1  32.392209
1     -0.024542 -0.118721       3        1  13.356302
2     -0.024821 -0.007305       4        1   7.119653
0     -0.011229 -0.003003       5        1   2.153386, topic_info=                Term       Freq      Total Category  logprob  loglift
377          product  33.000000  33.000000  Default  30.0000  30.0000
798  zhangjiang_tech   3.000000   3.000000  Default  29.0000  29.0000
46        investment  19.000000  19.000000  Default  28.0000  28.0000
626      real_estate  11.000000  11.000000  Default  27.0000  27.0000
180         security   4.000000   4.000000  Default  26.0000  26.0000
..               ...        ...        ...      ...      ...      ...
172      proprietary   0.034155   1.055625   Topic5  -6.8283   0.4072
321        hong_kong   0.056131   7.618229   Topic5  -6.3315  -1.0725
277            brand   0.045456   3.663993   Topic5  -6.5425  -0.5514
346         mainland   0.046479   4.459153   Topic5  -6.5202  -0.7256
377          product   0.047653  33.999020   Topic5  -6.4953  -2.7320

[252 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
480       1  0.699191        12th_plan
480       4  0.349596        12th_plan
467       3  0.623847      acquisition
132       3  0.626348          analyst
671       1  0.691658            anxin
...     ...       ...              ...
77        1  0.589374             week
779       1  0.690804              xin
7         2  0.988072            xindu
797       4  0.603538       zhangjiang
798       4  0.947301  zhangjiang_tech

[212 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 3, 1])

In [30]:
pyLDAvis.save_html(vis, 'liduo_lda.html')

## Topic Rank

In [31]:
vis.sorted_terms()

,Term,Freq,Total,Category,logprob,loglift,relevance
491,industry,11.099773,14.002698,Topic1,-4.0837,0.5667,-4.0837
377,product,8.906156,33.999020,Topic1,-4.3039,-0.5406,-4.3039
46,investment,8.217828,19.905523,Topic1,-4.3843,-0.0857,-4.3843
321,hong_kong,7.144752,7.618229,Topic1,-4.5242,0.7348,-4.5242
309,financial,5.723705,8.326729,Topic1,-4.7460,0.4241,-4.7460
350,medium,4.830633,5.267005,Topic1,-4.9156,0.7125,-4.9156
355,negative,4.069527,4.483418,Topic1,-5.0871,0.7021,-5.0871
310,flavor,4.053897,4.466264,Topic1,-5.0909,0.7021,-5.0909
346,mainland,4.038236,4.459153,Topic1,-5.0948,0.6998,-5.0948
455,investor,3.433960,7.454400,Topic1,-5.2569,0.0239,-5.2569


# 利空结果展示

## Visualization

In [32]:
vis = pyLDAvis.gensim_models.prepare(likong_model, likong_corpus, dictionary=likong_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.128695  0.069199       1        1  24.789379
3     -0.095924 -0.016456       2        1  23.752480
0     -0.093777  0.071177       3        1  21.180774
1      0.030731 -0.126498       4        1  17.757376
2      0.030276  0.002578       5        1  12.519991, topic_info=                Term      Freq     Total Category  logprob  loglift
383           banker  3.000000  3.000000  Default  30.0000  30.0000
218  shantou_lianmei  4.000000  4.000000  Default  29.0000  29.0000
117        oil_shale  3.000000  3.000000  Default  28.0000  28.0000
10             stock  4.000000  4.000000  Default  27.0000  27.0000
51             share  3.000000  3.000000  Default  26.0000  26.0000
..               ...       ...       ...      ...      ...      ...
35          internal  0.553258  2.820617   Topic5  -4.9880   0.4490
72          lobbyist  0.549363  1.652948   Topic5  -4.9950   0.9763
271         shanghai  0.549343  2.208682   Topic5  -4.9951   0.6864
334          opinion  0.549333  2.270597   Topic5  -4.9951   0.6588
45              term  0.549332  1.644147   Topic5  -4.9951   0.9816

[244 rows x 6 columns], token_table=      Topic      Freq                    Term
term                                         
280       1  0.836690                     20e
380       5  0.974163                  absorb
409       2  0.842908                acquired
152       2  0.554963             acquisition
152       3  0.277481             acquisition
...     ...       ...                     ...
378       5  0.389324               yesterday
70        4  0.893334                    yili
469       2  0.825874  yizhuang_international
18        4  0.893333                   zhang
233       3  0.862208             zhuangqiang

[242 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 2, 3])

In [33]:
pyLDAvis.save_html(vis, 'likong_lda.html')

## 主题词排序

# 合并结果展示

## Visualization

In [34]:
import pyLDAvis
from pyLDAvis import gensim_models
vis= gensim_models.prepare(merged_model, merged_corpus, dictionary=merged_dictionary)
#vis = pyLDAvis.gensim_models.prepare(merge_model, merge_corpus, dictionary=merge_dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.184088 -0.006548       1        1  27.714424
4      0.060142 -0.103824       2        1  22.722131
0      0.070746  0.152675       3        1  22.420922
3      0.068701 -0.076224       4        1  17.537419
1     -0.015501  0.033921       5        1   9.605103, topic_info=            Term       Freq      Total Category  logprob  loglift
377      product  20.000000  20.000000  Default  30.0000  30.0000
492     industry  15.000000  15.000000  Default  29.0000  29.0000
46    investment  18.000000  18.000000  Default  28.0000  28.0000
246      company   8.000000   8.000000  Default  27.0000  27.0000
83        banker   7.000000   7.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
144         data   1.245402   3.586408   Topic5  -5.0878   1.2852
158      holding   1.207394   4.052586   Topic5  -5.1188   1.1320
331  institution   1.065167   3.128908   Topic5  -5.2442   1.2653
567         fund   1.207889   6.692880   Topic5  -5.1184   0.6307
199       reveal   0.942812   2.631740   Topic5  -5.3662   1.3163

[228 rows x 6 columns], token_table=      Topic      Freq                    Term
term                                         
481       3  0.692413               12th_plan
468       3  0.974572             acquisition
241       3  0.692827                  annual
672       3  0.801908                   anxin
946       1  0.678906                 article
...     ...       ...                     ...
450       5  0.543626               yesterday
886       5  0.686649                    yili
1172      4  0.878256  yizhuang_international
799       3  0.802811              zhangjiang
800       3  0.804720         zhangjiang_tech

[238 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 4, 2])

In [35]:
pyLDAvis.save_html(vis, 'merged_lda.html')

## 主题词排序

In [36]:
vis.sorted_terms()

,Term,Freq,Total,Category,logprob,loglift,relevance
986,shantou_lianmei,8.183730,8.733610,Topic1,-4.2648,1.2182,-4.2648
377,product,7.400232,20.994352,Topic1,-4.3654,0.2405,-4.3654
350,medium,5.046540,5.548317,Topic1,-4.7482,1.1884,-4.7482
1092,sector_dragon,4.701836,5.250817,Topic1,-4.8190,1.1728,-4.8190
1093,shipment,4.701836,5.250817,Topic1,-4.8190,1.1728,-4.8190
263,report,4.000798,6.136978,Topic1,-4.9805,0.8554,-4.9805
309,financial,3.977590,7.481087,Topic1,-4.9863,0.6515,-4.9863
321,hong_kong,3.698268,4.189245,Topic1,-5.0591,1.1586,-5.0591
981,profile,3.585912,4.101862,Topic1,-5.0899,1.1488,-5.0899
1046,brother,3.563122,4.096135,Topic1,-5.0963,1.1438,-5.0963
